# UNDP Development Minerals 2 Illegal River Extraction Monitoring

<u>Vectors</u>: \
**Vegetation Change Detection**:      Normalised Difference Vegetation Index (NDVI), Agriculture \
**River/Waterways Change Detection**: Normalised Difference Water Index (NDWI)  \
**Turbidity/Sediment Index**:         Normalised Difference Turbidity Index (NDTI) \

<u>Indicies</u>: \
**NDVI**: (NIR - RED) / (NIR + RED), (B08 - B04) / (B08 + B04) \
**NDWI**: (GREEN - NIR) / (GREEN + NIR) >= 0.3 - Water, < 0.3 - Non-water, (B8A - B11) / (B8A + B11) \
**NDTI**: (RED - GREEN) / (RED + GREEN) , (B04 - B03) / (B04 + BO3) \
**Agriculture**: B11, B08, B02 (SWIR16, NIR, BLUE) 


In [ ]:
!pip install -q leafmap

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import leafmap
import dask
from dask_gateway import GatewayCluster
from pystac_client import Client
import planetary_computer as pc
import stackstac
import numpy as np
import xarray as xr
import xrspatial.multispectral as ms

### Area and Time Of Interest

In [ ]:
local =  gpd.read_file('fiji_river_dawasamu.geojson')
time_range = '2022-01-01/2022-12-31'

area_of_interest = local.geometry[0]
bbox = local.total_bounds
local.explore()

### Setup Dask Cluster

In [ ]:
cluster = GatewayCluster()  # Creates the Dask Scheduler. Might take a minute.
client = cluster.get_client()
cluster.adapt(minimum=4, maximum=100)
print(cluster.dashboard_link)

### Request Imageries (STAC)

In [ ]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest, 
    datetime=time_range,
    limit=500,  
    query={"eo:cloud_cover": {"lt": 10}},
)
items = list(search.get_items())
for item in items:
    print(f"{item.id}: {item.datetime}")
print(f"{len(items)} Images Returned")

### Sign Images

In [ ]:
items = list(search.get_items())
items = [pc.sign(item).to_dict() for item in items]

### Stack Images Into Data (xarray)

In [ ]:
data = ( 
        stackstac.stack(items, 
                        bounds_latlon=bbox,
                        assets=["B04", "B03", "B02", "B08", "B11"],  # red, green, blue, nir, swir16
                        chunksize=8124,
                        resolution=500
                       )
        .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
        .assign_coords(
            band=lambda x: x.common_name.rename("band"), # use common names
            time=lambda x: x.time.dt.round("D")
        )  
       )
data

### Define Bands

In [ ]:
red = data.sel(band="red")
blue = data.sel(band="blue")
green = data.sel(band="green")
nir = data.sel(band="nir")
swir = data.sel(band="swir16")

### Vector: Agriculture Band Composite

In [ ]:
vegetation = data.sel(band=["swir16", "nir", "blue"])

In [ ]:
vegetation.plot.imshow(x="x", y="y", col="time", col_wrap=5, cmap="viridis")#.compute()

### Vector: NDVI (Vegetation Index)

In [ ]:
ndvi = ((nir - red) / (red + nir)).compute()
#fig, ax = plt.subplots(figsize=(12, 12))
#ndvi.plot.imshow(cmap="viridis", vmin=-0.6, vmax=0.6, add_colorbar=False, ax=ax)
#ax.set_axis_off()

In [ ]:
ndvi.plot.imshow(x="x", y="y", col="time", col_wrap=5, cmap="viridis")

### Vector: NDWI (Water Index)

In [ ]:
ndwi = ((green - nir) / (green + nir)).compute()
#remove values less then 0.3

In [ ]:
ndwi.plot.imshow(x="x", y="y", col="time", col_wrap=5, cmap="viridis")

### Vector: NDTI (Turbidity Index)

In [ ]:
ndti =  ((red - green) / (red + green)).compute()

In [ ]:
ndti.plot.imshow(x="x", y="y", col="time", col_wrap=5, cmap="viridis")

### Change Detection Threshold Definition 
Identification of AOI and DateTime of Extraction

In [ ]:
ndvi_change_percent = 0.45
ndwi_change_percent = 0.35
ndti_change_percent = 0.70
vegetation_change_percent = 0.25

In [ ]:
#TO BE IMPLEMENTED: calculating and aligning thresholds of the 4 vectors